kickstarter 홈페이지에서 프로젝트의 이름을 가져오기에는 여러 문제점(구글 독스 참고)이 있어서 kicktraq 홈페이지에서 가져오기로 한다.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns',999)
pd.set_option('max_rows',200)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
sns.set() 
%matplotlib inline 
%config InlineBackend.figure_format = 'retina' 
sns.set_style(style='white')
sns.set_context(context ='paper')

import datetime
import os
from functools import reduce

In [2]:
import selenium as se
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re

In [3]:
driver = wd.Chrome(r'C:\Users\jeje9\Downloads//chromedriver.exe')

In [4]:
# 크롤링 결과 저장할 df
blurb_df = pd.DataFrame(columns = ['blurb', 'state'])

### >> 성공한 프로젝트 목록 가져오기
우측에 있는 day-1 Projects에 성공한 프로젝트가 많아서 이 카테고리에서 가져오려고 함...

day-1 카테고리 설명 >> 'Day-1' projects are campaigns we have been tracking since the project received their very first backer! We give these projects their own special category because their project owners or backers took the time to add them to Kicktraq on their first day. You took the time for us, we took the time for you!

### 문제가 있다!!!
각 페이지마다 보여지는 프로젝트의 숫자가 달라서 for문을 돌리기 어렵다 --> 그래도 각 페이지 별로 최소 5개~7개 정도는 뜰테니 다 하지 말고 이정도만 한다.

In [ ]:
#project-list > div.projects > div:nth-child(3) > div.project-infobox > div.project-cat > a:nth-child(1)

In [ ]:
#project-list > div.projects > div:nth-child(7) > div.project-infobox > div.project-cat > a

In [ ]:
#project-list > div.projects > div:nth-child(8) > div.project-infobox > div.project-cat

In [5]:
# 1페이지부터 5페이지까지
for page in range(1, 6): 
    driver.get('https://www.kicktraq.com/dayones/?page={}'.format(page))
    
    # 페이지의 첫 번째에 뜨는 프로젝트는 css_selector가 달라서 따로 처리
    detail = driver.find_element_by_css_selector('#project-list > div.projects > div.project.after-ribbon.odd > div.project-infobox > div.project-infobits > div.project-details').text
    percent = int(detail.split("\n")[1].split('(')[1].split('%')[0])
    
    if percent >= 100:
        blurb = driver.find_element_by_css_selector('#project-list > div.projects > div.project.after-ribbon.odd > div.project-infobox > div:nth-child(2)').text
        print(blurb)
        # blurb 없는 애들 빼기
        if blurb != '':
            row = [blurb, 1]
            blurb_df = blurb_df.append(pd.Series(row, index=['blurb','state']), ignore_index=True)
    
    # 각 페이지의 두 번째 프로젝트부터 9개
    for i in range(3, 12):
        detail = driver.find_element_by_css_selector('#project-list > div.projects > div:nth-child({}) > div.project-infobox > div.project-infobits > div.project-details'.format(i)).text
        percent = int(detail.split("\n")[1].split('(')[1].split('%')[0])
    
        if percent >= 100:
            blurb = driver.find_element_by_css_selector('#project-list > div.projects > div:nth-child({}) > div.project-infobox > div:nth-child(2)'.format(i)).text
            print(blurb)
            # blurb 없는 애들 빼기
            if blurb != '':
                row = [blurb, 1]
                blurb_df = blurb_df.append(pd.Series(row, index=['blurb','state']), ignore_index=True)

A high-quality screen-printed tee featuring artwork by Matthew Warlick.
Produce a 2D animated short film featuring music by 9 Theory
Taskin Air Duo has dual-side compression packing so you can pack more in the same space and keep clean & dirty clothes separate
Get rid of nickel and chromium leached from steel cutlery. Woerden Cutlery is made of solid pure Titanium. Safe + Hygienic!
Give your “stubborn” iPhone the trigger buttons as console gamepads do. Engineered for mobile gaming aficionados, GameSir i3 can do.
Monday is delicious, alcohol free gin. Enjoy the drinks you love sans the hangover you hate.
A simple, innovative faucet attachment to save water
Help me fund the production of this little marshmallow floof enamel pin ♥
Follow aspiring actor Stu's quirky life in the city of dreams: London. A new viewing experience merging TV Series and Social Media.
Tori's gone wild with her horoscope cosplays, here are the pins she's crafted from them.
A decadent assortment of silly snakes and

------------------------

### >> 실패한 프로젝트 목록 가져오기
우측에 있는 Archived Projects에 실패한 프로젝트가 많아서 이 카테고리에서 가져오려고 함...

### 문제가 있다!!!
1 page부터 보면 프로젝트 진행 날짜가 아직 많이 남은 live 프로젝트가 뜨게 됨. --> 대략 18 page부터 보면.... 이미 끝난 프로젝트를 확인할 수 있음...

In [6]:
# 18페이지부터 22페이지까지 총 5 페이지
for page in range(18, 23): 
    driver.get('https://www.kicktraq.com/archive/?page={}'.format(page))
    
    # 페이지의 첫 번째에 뜨는 프로젝트는 css_selector가 달라서 따로 처리
    detail = driver.find_element_by_css_selector('#project-list > div.projects > div.project.first.odd.closed > div.project-infobox > div.project-infobits > div.project-details').text
    percent = int(detail.split("\n")[1].split('(')[1].split('%')[0])
    
    if percent < 100:
        blurb = driver.find_element_by_css_selector('#project-list > div.projects > div.project.first.odd.closed > div.project-infobox > div:nth-child(2)'.format(i)).text
        print(blurb)
        # blurb 없는 애들 빼기
        if blurb != '':
            row = [blurb, 0]
            blurb_df = blurb_df.append(pd.Series(row, index=['blurb','state']), ignore_index=True)
    
    # 각 페이지의 두 번째 프로젝트부터 9개
    for i in range(3, 12):
        detail = driver.find_element_by_css_selector('#project-list > div.projects > div:nth-child({}) > div.project-infobox > div.project-infobits > div.project-details'.format(i)).text
        percent = int(detail.split("\n")[1].split('(')[1].split('%')[0])
    
        if percent < 100:
            blurb = driver.find_element_by_css_selector('#project-list > div.projects > div:nth-child({}) > div.project-infobox > div:nth-child(2)'.format(i)).text
            print(blurb)  
            # blurb 없는 애들 빼기
            if blurb != '':
                row = [blurb, 0]
                blurb_df = blurb_df.append(pd.Series(row, index=['blurb','state']), ignore_index=True)

Handcrafted, artisanal soaps made with love and other fragrances.
Record CDs of my harp and piano playing.
Challenge gravity and push yourself at the edge of physical laws in order to mantain an impossible trajectory
10 Minute Space Battle. A tactical dexterity card game for 2 to 6 space pilots with a few minutes to spare. Easy as rock-paper-railgun!
an online mobile travel planner and social networking service
Turn your treadmill or stationary bicycle into a gaming machine! Have fun and get motivated to get fit!
A standard 54-card pack featuring Marvel villains.
A small secure device to be placed outside of the home where packages can be delivered and secured with and audible alarm system.
A helpful guide to fulfill your sexual partner to maximum pleasure.
An interview-style podcast & website exploring the history and current lives of LGBT Australians, starting with Tasmania.
Colouring your hair is as simple as running a comb across your locks.
Tiny modern off-grid sustainable cottage

### 결과 DF

In [7]:
blurb_df

,blurb,state
0,A high-quality screen-printed tee featuring ar...,1
1,Produce a 2D animated short film featuring mus...,1
2,Taskin Air Duo has dual-side compression packi...,1
3,Get rid of nickel and chromium leached from st...,1
4,Give your “stubborn” iPhone the trigger button...,1
5,"Monday is delicious, alcohol free gin. Enjoy t...",1
6,"A simple, innovative faucet attachment to save...",1
7,Help me fund the production of this little mar...,1
8,Follow aspiring actor Stu's quirky life in the...,1
9,"Tori's gone wild with her horoscope cosplays, ...",1


In [ ]:
data = pd.read_csv('../df_text_eng.csv')
data = data.drop(['Unnamed: 0'], axis = 1)